In [ ]:
from stageOne import convert
import pandas as pd
import os 
from bs4 import BeautifulSoup
import re 
import matplotlib.pyplot as plt
import numpy as np

def scatter(dates,freqs,title,color,x_label,y_label):
    xData = list(map(int, dates))
    yData = list(map(int, freqs))
    
    xyCoords=np.c_[xData,yData]
    
    uniquePoints, counts = np.unique(xyCoords, return_counts=True,axis=0)
    plt.scatter(uniquePoints[:,0],uniquePoints[:,1],s=counts*10,c=color,)

    fig = plt.gcf()
    fig.set_size_inches(15, 10)

    plt.xlabel(x_label, fontsize=20)
    plt.ylabel(y_label, fontsize=20)
    
    plt.title(title,fontsize=25)
    
    plt.show()

def bar(xData,yData,title,xlabel,ylabel):
    if (xData == "Year"):
        xData = list(map(int, xData))
        yData = list(map(int, yData))
    
    plt.figure(figsize = (15, 10))

    plt.bar(xData,yData,color='purple')
    
    plt.title(title, fontsize=20)
    
    plt.xlabel(xlabel, fontsize=15)
    plt.ylabel(ylabel, fontsize=15)
    
    plt.show()

In [ ]:
dir = '/srv/data/'
f1 = 'eebo_phase1_IDs_and_dates.txt'
f2 = 'EEBO_Phase2_IDs_and_dates.txt' 

def getDates(folder,f,dRanges,unknown,ourTimes):
    file = os.path.join(folder,f)
    data = open(file,'r')
    data = data.readlines()
    dates = {}
    for d in data:
        datum = d.replace('\n','')
        datum = d.split('\t')
        id = datum[0]
        date = datum[1]
        
        if re.search('u', date):
            unknown[id] = date
            continue
        
        if date.find('?') != -1:
            unknown[id] = date
            continue

        if re.search('-',date):
            r = date.split('-')
            dRanges[id] = r
            date = dRanges[id][0]
            continue

        value = int(date)
        # if 1580 <= value or value <= 1640:
        if value in range(1580,1640+1):
            ourTimes[id] = value
            if value not in dates.keys():
                dates[value] = []
            else:
                dates[value].append(id)
    print("There are ",len(ourTimes), " texts in your date range, ", len(dRanges), " texts with date ranges, and ", len(unknown), " texts with unknown dates, marked with a 'u'.")
    # print(f"Here are the texts within your range: {ourTimes}")
    # print(f"Here are the texts with a time range: {dRanges}")
    # print(f"These are the texts with unknown dates: {unknown}")
    return dates

def plotFreqs(dList):
    freqs = {}
    for dict in dList: 
        for key in dict.keys():
            if key not in freqs.keys():
                freqs[key] = len(dict[key])
            else: 
                freqs[key] += len(dict[key])
    scatter(freqs.keys(),freqs.values(),"TCP Texts Per Year","purple","Year","Number of TCP Texts")
    bar(freqs.keys(),freqs.values(),"TCP Texts Per Year","Year","Number of TCP Texts")

In [ ]:
d1 = getDates(dir,f1,{},{},{})
d2 = getDates(dir,f2,{},{},{})

plotFreqs([d1,d2]) 

